In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from transformers import RobertaTokenizer


In [2]:
def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer, orig_answer_text):
    """Returns tokenized answer spans that better match the annotated answer."""
    tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))

    for new_start in range(input_start, input_end + 1):
        for new_end in range(input_end, new_start - 1, -1):
            text_span = " ".join(doc_tokens[new_start : (new_end + 1)])
            if text_span == tok_answer_text:
                return (new_start, new_end)

    return (input_start, input_end)

In [3]:
train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = RobertaTokenizer.from_pretrained('../../bert_models/roberta_base/')

In [5]:
train.shape

(27481, 4)

In [6]:
train.dropna(subset=['text','selected_text'], how='any', inplace=True)

In [7]:
train.shape

(27480, 4)

In [8]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [9]:
train['text'] = train['text'].apply(lambda x: ' '.join(x.strip().split()))
train['selected_text'] = train['selected_text'].apply(lambda x: ' '.join(x.strip().split()))

In [10]:
train['start_pos'] = train.apply(lambda x: x['text'].find(x['selected_text']), axis=1)
train['end_pos'] = train.apply(lambda x: x['start_pos']+len(x['selected_text']), axis=1)

In [11]:
train['start_pos'].describe()

count    27480.000000
mean        15.510153
std         26.048759
min          0.000000
25%          0.000000
50%          0.000000
75%         23.000000
max        133.000000
Name: start_pos, dtype: float64

In [12]:
def contains(a, b):
    for i in range(0, len(a)-len(b)+1):
        flag = True
        for j in range(0, len(b)):
            if a[i+j]!=b[j]:
                flag = False
                break
        if flag:
            return True
    return False

In [13]:
train['text_tokens'] = train['text'].apply(lambda x: tokenizer.tokenize(' '+x))
train['st_tokens'] = train['selected_text'].apply(lambda x: tokenizer.tokenize(' '+x))

In [14]:
data = []
improve_count = 0
for text_token, st_token in zip(train['text_tokens'].tolist(), train['st_tokens'].tolist()):
    list_contain = 1 if contains(text_token, st_token) else 0
    tokens = ' '.join(text_token)
    st_tokens = ' '.join(st_token)
    str_contain = 1 if tokens.find(st_tokens)>=0 else 0
    data.append((list_contain, str_contain))

In [15]:
list_contain, str_contain = zip(*data)

In [16]:
sum(list_contain)/len(list_contain)

0.950254730713246

In [44]:
sum(str_contain)/len(str_contain)

0.9382096069868996

In [48]:
improve_count

1565

In [49]:
tokenizer.tokenize("bullying me...")

['bullying', 'me', '.', '.', '.']

In [50]:
tokens, start, end, invert_map, _, _, not_match, first_token = zip(*data)

In [51]:
len(tokens)

27480

In [52]:
train['tokens'] = tokens

In [53]:
train['start'] = start
train['end'] = end
train['invert_map'] = invert_map
train['first_token'] = first_token

In [54]:
senti2label = {
    'positive':2,
    'negative':0,
    'neutral':1
}
train['senti_label']=train['sentiment'].apply(lambda x: senti2label[x])

In [55]:
train.reset_index(drop=True, inplace=True)

In [56]:
train.head()

,textID,text,selected_text,sentiment,start_pos,end_pos,tokens,start,end,invert_map,senti_label,fold,first_token
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,0,35,"[i, `, d, have, responded, ,, if, i, were, going]",0,9,"[0, 0, 0, 1, 2, 2, 3, 4, 5, 6]",1,0,"[True, False, False, True, True, False, True, ..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,0,8,"[soo, ##o, sad, i, will, miss, you, here, in, ...",0,2,"[0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 9]",0,0,"[True, False, True, True, True, True, True, Tr..."
2,088c60f138,my boss is bullying me...,bullying me,negative,11,22,"[my, boss, is, bullying, me, ., ., .]",3,4,"[0, 1, 2, 3, 4, 4, 4, 4]",0,0,"[True, True, True, True, True, False, False, F..."
3,9642c003ef,what interview! leave me alone,leave me alone,negative,16,30,"[what, interview, !, leave, me, alone]",3,5,"[0, 1, 1, 2, 3, 4]",0,0,"[True, True, False, True, True, True]"
4,358bd9e861,"Sons of ****, why couldn`t they put them on th...","Sons of ****,",negative,0,13,"[sons, of, *, *, *, *, ,, why, couldn, `, t, t...",0,6,"[0, 1, 2, 2, 2, 2, 2, 3, 4, 4, 4, 5, 6, 7, 8, ...",0,0,"[True, True, True, False, False, False, False,..."


In [57]:
from sklearn.model_selection import StratifiedKFold

In [58]:
kf = StratifiedKFold(n_splits=5)

In [59]:
train['fold'] = 0
for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train['senti_label'])):
    train.loc[valid_idx, 'fold'] = fold

In [60]:
train['fold'].value_counts()

0    5498
1    5497
4    5495
3    5495
2    5495
Name: fold, dtype: int64

In [61]:
train.to_pickle('../input/train.pkl')